In [ ]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from tsfresh import extract_features
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
import torch
from Model import Sample_Attention_LSTM
from data import Sample_Attention_LSTM_dataset
from train import train
from torch.utils.data import DataLoader
import seaborn as sns
import numpy as np

In [ ]:
df = pd.read_excel('C:/Users/Administrator/Desktop/科研/github/Data/202209高压用户出账电量.xlsx')

In [ ]:
df_id = df["户号"]
df_ht = df["合同容量"]
df_yx = df["运行容量"]
df_ek = df["用电类别"]
df_level = df["电压等级"]
df_pm = df["电价码"]
df_ld = df["临电标志"]
df_val = df.iloc[:,23:]

In [ ]:
#one_hot
df_ek = pd.get_dummies(df_ek)
df_level = pd.get_dummies(df_level)
df_pm = pd.get_dummies(df_pm)
df_ld = pd.get_dummies(df_ld)

In [ ]:
#统计指标抽取
df_val = df_val.fillna(0)
ts_data = df_val.stack().reset_index()
ts_data.columns = ['id', 'time', 'value']
# 使用 tsfresh 抽取特征
extracted_features = extract_features(ts_data, column_id='id', column_sort='time', column_value='value')
# extracted_features = df_val

In [ ]:
# combined_data 维度(7882, 1022)
combined_data = pd.concat([df_val, extracted_features,df_ek,df_level,df_pm,df_ld,df_ht,df_yx], axis=1)
combined_data = combined_data.fillna(0)
# 创建 StandardScaler 实例
scaler = StandardScaler()

# 使用 StandardScaler 对 combined_data 进行标准化
standardized_data = pd.DataFrame(scaler.fit_transform(combined_data), columns=combined_data.columns)
# 应用 PCA 降维
pca = PCA(n_components=0.5)
reduced_features = pca.fit_transform(standardized_data)
# 显示结果
print("抽取统计特征之后的特征数：", standardized_data.shape[1])
print("降维后的特征数：", reduced_features.shape[1])

In [ ]:
# 设置要尝试的聚类数目范围
cluster_range = range(2, 11)

# 用于存储每个聚类数目的轮廓系数和簇内误方差
silhouette_scores = []
inertias = []

# 对于每个聚类数目，进行k-means++聚类并计算轮廓系数和簇内误方差
for n_clusters in cluster_range:
    kmeans = KMeans(n_clusters=n_clusters, init='k-means++', random_state=42)
    kmeans.fit(reduced_features)
    labels = kmeans.labels_
    silhouette_scores.append(silhouette_score(reduced_features, labels))
    inertias.append(kmeans.inertia_)

# 绘制轮廓系数图 轮廓系数处于[-1,1]之间，越高越好
plt.figure(figsize=(12, 6))
plt.plot(cluster_range, silhouette_scores, marker='o', linewidth=2)
plt.xlabel("Number of clusters", fontsize=14)
plt.ylabel("Silhouette Score", fontsize=14)
plt.title("Silhouette Score vs Number of Clusters", fontsize=16)
plt.grid(True)
plt.show()

# 绘制簇内误方差图 找肘部点，即误差下降速度突然减缓的点，这个点通常代表了一个合适数目的聚类
plt.figure(figsize=(12, 6))
plt.plot(cluster_range, inertias, marker='o', linewidth=2)
plt.xlabel("Number of clusters", fontsize=14)
plt.ylabel("Inertia", fontsize=14)
plt.title("Inertia vs Number of Clusters", fontsize=16)
plt.grid(True)
plt.show()

In [ ]:
#由上图可以看出，适合聚3类
kmeans = KMeans(n_clusters=3, init='k-means++', random_state=42)
kmeans.fit(reduced_features)
labels = kmeans.labels_

In [ ]:
# 确保 df_id 和 labels 的长度相同
assert len(df_id) == len(labels), "Lengths of df_id and labels do not match."

# 创建一个新的 DataFrame，将 df_id 转换为 DataFrame
df_id_df = pd.DataFrame({'id': df_id.values})

# 将聚类标签添加到新的 DataFrame
df_id_df['cluster'] = labels

# 重置索引
df_id_df.reset_index(drop=True, inplace=True)

# 按照 cluster 标签将用户分组
cluster_0 = df_id_df[df_id_df['cluster'] == 0]
cluster_1 = df_id_df[df_id_df['cluster'] == 1]
cluster_2 = df_id_df[df_id_df['cluster'] == 2]

In [ ]:
df_id_val = pd.concat([df_id,df_val],axis=1)
# 从原始 DataFrame 中获取 cluster_0 的数据
cluster_0_data = df_id_val[df_id_val['户号'].isin(cluster_0['id'])]

# 从原始 DataFrame 中获取 cluster_1 的数据
cluster_1_data = df_id_val[df_id_val['户号'].isin(cluster_1['id'])]

# 从原始 DataFrame 中获取 cluster_2 的数据
cluster_2_data = df_id_val[df_id_val['户号'].isin(cluster_2['id'])]

In [ ]:
odd_indices = list(range(1, 138, 2))
cluster_0_sum = cluster_0_data.iloc[:,odd_indices].sum(axis=0)
cluster_0_sum = cluster_0_sum.to_numpy()

cluster_1_sum = cluster_1_data.iloc[:,odd_indices].sum(axis=0)
cluster_1_sum = cluster_1_sum.to_numpy()

cluster_2_sum = cluster_2_data.iloc[:,odd_indices].sum(axis=0)
cluster_2_sum = cluster_2_sum.to_numpy()
cluster_2_sum = torch.tensor(cluster_2_sum)

In [ ]:
#选择最后一类的数据作为样例
plt.figure(figsize=(20,6))
plt.plot(cluster_2_sum)
plt.title("Cluster 2 sum Curve")
plt.xlabel("time")
plt.ylabel("sum")
plt.show()

In [ ]:
seq_len = 12
horizon_size = 1

train_data = cluster_2_sum[0:50]
test_data = cluster_2_sum[50:]

# 计算张量的均值和标准差
mean = train_data.mean(dim=0)
std = train_data.std(dim=0)
# 对张量进行标准化
train_data = (train_data - mean) / std

attention_key = []
for i in range(0,len(train_data)-seq_len-horizon_size):
    input_seq = train_data[i:i+seq_len]
    attention_key.append(input_seq)
attention_key = torch.stack(attention_key)

attention_value = []
for i in range(0,len(train_data)-seq_len-horizon_size):
    input_value = train_data[i:i+seq_len+horizon_size]
    attention_value.append(input_value)
attention_value = torch.stack(attention_value)


In [ ]:
#导入模型
config = {
    'horizon_size':1,
    'hidden_size':8,#8
    'dropout': 1e-1,
    'layer_size':1,
    'lr': 1e-2,#5
    'batch_size': 10,
    'num_epochs':60,
    'L1':4,
    'L2':2,
    'seq_len':12,
    
}
horizon_size = config['horizon_size']
hidden_size = config['hidden_size']
dropout = config['dropout']
layer_size = config['layer_size']
seq_len = config['seq_len']
lr = config['lr']
batch_size = config['batch_size']
num_epochs = config['num_epochs']
L1 = config['L1']
L2 = config['L2']

myseed = 6666  # set a random seed for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(myseed)
torch.manual_seed(myseed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(myseed)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
dataset = Sample_Attention_LSTM_dataset(train_data,horizon_size,seq_len)
model = Sample_Attention_LSTM(horizon_size,seq_len,dropout,layer_size,L1,L2,hidden_size,device)



In [ ]:
train(model,dataset,lr,batch_size,num_epochs,attention_key,attention_value,device)

In [ ]:
#绘制拟合曲线
sample_nums = len(train_data)-horizon_size-seq_len
data_iter = DataLoader(dataset=dataset, batch_size=sample_nums, shuffle=False,num_workers=0)
for (sample_input,sample_vals) in data_iter:
    fit_seq = model.predict(sample_input,attention_key=attention_key,attention_value=attention_value)
    fit_seq = fit_seq * std.reshape(-1, 1) + mean.reshape(-1, 1)
    sample_vals = sample_vals * std.reshape(-1, 1) + mean.reshape(-1, 1)
    plt.plot(fit_seq,color = '#1f77b4',label='fit_seq')
    plt.plot(sample_vals, color = '#d62728',label='real_val')
    plt.legend()

In [ ]:
#绘制测试集预测曲线
# 对张量进行标准化
test_mean = test_data.mean(dim=0)
test_std = test_data.std(dim=0)
test_data_n = (test_data - test_mean) / test_std

In [ ]:
test_dataset = Sample_Attention_LSTM_dataset(test_data_n,horizon_size,seq_len)
test_sample_nums = len(test_data_n)-seq_len-horizon_size
test_data_iter = DataLoader(dataset=test_dataset, batch_size=test_sample_nums, shuffle=False,num_workers=0)
for (test_input,test_real_vals) in test_data_iter:
    print(test_input.shape)
    test_seq = model.predict(test_input,attention_key=attention_key,attention_value=attention_value)
    print(test_seq.shape)
    test_seq = test_seq * test_std.reshape(-1, 1) + test_mean.reshape(-1, 1)
    test_real_vals = test_real_vals * test_std.reshape(-1, 1) + test_mean.reshape(-1, 1)
    plt.plot(test_seq,color = '#1f77b4',label='predict_seq')
    plt.plot(test_real_vals, color = '#d62728',label='real_val')
    plt.legend()

In [ ]:
#可以看到，效果还是很不错的，就是有一个异常点误差比较大34%
#经过历史数据的查询，发现该时间点对应的是2022年4月，正好是疫情影响最严重的时间，与2021年相比，下挫了25%
#如果按照正常规律发展，应该比往年略微有所上升
#因此，如果不考虑疫情的影响，该点误差应该会落在10%以内
#总误差2.67%
mape = (test_real_vals - test_seq)/test_real_vals
print(mape)
s_mape = (sum(test_real_vals) - sum(test_seq))/sum(test_real_vals)
print(s_mape)

In [ ]:
print(test_real_vals)

In [ ]:
sns.set_style('whitegrid')

plt.figure(figsize=(12,6))
sns.lineplot(x=np.arange(len(mape)), y=np.array(mape).reshape(-1), marker='o', markersize=10, linewidth=2)
plt.xlabel('data points', fontsize=14)
plt.ylabel('MAPE', fontsize=14)
plt.title('Image of MAPE', fontsize=16)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.show()